> DUPLICATE THIS COLAB TO START WORKING ON IT. Using File > Save a copy to drive.

# Reference: State-of-the-art (SOTA) Supervised Learning Models

To help you get started with state-of-the-art ML modeling approaches across many domains,  we will be introducing a variety of model classes you can try on challenging ML tasks. We won't cover deep learning approaches in this project, but many of the best practices and classifier tricks covered here can help improve even complex deep learning systems. As usual, advanced students are welcome to dig deeper into the advanced algorithmic ideas covered here.

### Instructions

1. This notebook is simply a reference on advanced modeling ideas like Boosting and Bagging. Reading through these ideas in depth **is optional**. You can use only the summary below to see scikit models you can try to leverage these ideas in practice.

# Summary: SOTA Machine Learning Approaches


You may be surprised to know that most Kaggle competitions are not won by deep neural networks. Instead, most are won by classic machine learning methods combined with clever featurization. [Here](https://www.ednetchallenge.ai/aied-challenge-1) is a good example: Riiid is a Korean education company that released a Kaggle competition along with a 1M row dataset of student answering academic questions. Their goal was to produce a ML model to understand student knowledge that could predict whether students would correctly answer new (unseen) questions. Although many deep neural network approaches were tried, many [top performing solutions](https://www.kaggle.com/ragnar123/riiid-model-lgbm) used classic machine learning methods (with features derived from deep neural networks), in particular a method called Gradient Boosting, which we cover below.

Our goal in this section is introduce the main ideas behind SOTA models, and how to use these to produce SOTA results using the guiding themes of this course. We will introduce some key statistical concepts which tend to improve the performance of many model types: **Ensembling**, **Bagging**, and **Boosting**. These concepts combine with some model primitives you've already seen (linear models and decision trees) to form SOTA techniques for supervised ML.


## SOTA models to try in practice
When achieving the best possible dev/test set performance is critical, just about any ML model can improve by "merging" multiple versions of a model.

We cover three ways to "merge" models by combining different models together: ensembling, bagging, and boosting. Each of these have their individual strengths and weaknesses, and of course they can be combined. You can _ensemble boosted bagged models_ (what a mouthful!). While these are general statistical ML tools, usually we can rely on pre-built models which leverage these ideas for us.

Here are some of the most popular algorithms which leverage and combine these techniques in different ways. The scikit documentation gives more detail on what these models do, and available hyper-parameters to tune when using these models.

**TL;DR: You can try the models below on a range of classification and regression tasks, and often achieve the best possible modeling performance for that task. Because these models combine multiple versions of the same underlying model primitive, they are high variance and prone to over-fitting if not tuned properly.**

_Tier 1. Models that often win competitions. Fairly robust across datasets and easy to apply in practice:_
* Random forests (`scikit.ensemble.RandomForestClassifier/Regressor`): we implemented a simple version of a random forest. The actual one chooses a random subset of input features for each decision tree to increase diversity across trees.
* AdaBoost (`scikit.ensemble.AdaBoostClassifier/Regressor`): Remember, this boosting algorithm reweights training examples that previous models struggled with when training new models.
* Gradient Boosting (`scikit.ensemble.GradientBoostingClassifier/Regressor`): Recall that gradient boosting trains successive models to reduce the residual error made by previous models.
* XGboost (https://xgboost.readthedocs.io/en/stable): This is a popular gradient boosting library on top of decision trees. It is highly optimized and known to have good performance. Unfortunately, it is not in scikit-learn but it is fairly easy to use. To install this, try `!pip install xgboost`. Read its documentation to see its syntax.

_Tier 2. Advanced models worth trying but slightly less popular/robust in practice compared to the above:_
* Voting (`scikit.ensemble.VotingClassifier/Regressor`): This implementation lets to pick between using the majority vote to make a prediction vs using average predicted probabilities (ie.e., a soft voting strategy).
* Combining strategies (`scikit.ensemble.StackingClassifier/Regressor`): This is a generic data structure to combine ensembling techniques together. Just remember to be careful of overfitting if you use this class.
* Extremely Randomized Trees (`scikit.ensemble.ExtraTreesClassifier`): This is a close variant of random forests. Like random forests, each decision tree is trained on a random subset of features. But unlike random forests, each decision tree is not trained to split the training data perfectly; rather some randomness is added in to increase diversity.

* Deep neural networks: Later courses will cover deep neural networks, which are often SOTA when large training sets are available, but require special optimization and modeling considerations.

Overall, if you're using techniques like the above and applying some hyper-parameter tuning to achieve good regularization settings, you will likely produce near SOTA models on most tasks. There is a dependence on datasets and the input/output function being modeled, but these techniques perform well in many production ML systems.

# Preliminaries

## Dependencies

We first setup required libraries. Many of these may already be installed by default in Colab.

In [ ]:
!pip install numpy
!pip install scikit-learn
!pip install xgboost
!pip install librosa

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb  # gradient boosting library
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

## Demo Dataset

To showcase these model techniques, we will use a popular toy dataset containing patient features of a digitized image of a fine needle aspirate of a breast mass cell nuclei. The task is to predict if the cell is benign or malignant. This dataset is publicly available on scikit-learn.

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

dataset = datasets.load_breast_cancer()
X = dataset.data
y = dataset.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(455, 30) (455,)
(114, 30) (114,)


# Ensembling: Combining Model Diversity

Supervised learning algorithms try to fit a model that performs well on a prediction problem. However, for complex prediction problems, there may be multiple solutions. For example, we use [ensembling](https://en.wikipedia.org/wiki/Ensemble_learning) in the form of random forest classifiers. Ensembling takes a collection of diverse models that solve the same prediction problem into a single model with potentially better performance.

**Deeper into the random forest**

In Week 1, we played with a decision tree, which splits the feature space into partitions that are mapped to target labels. However, the partitioning is a somewhat random process, and fitting the decision tree with different random seeds can result in very different partitions.

A random forest is an ensemble of decision trees. It trains multiple decision trees independently. Then, to make a prediction on a new example, each decision tree makes a prediction as a "vote", and the forest spits out the the majority vote. For example, if the ensemble has 10 trees and 8 of them predict a positive label whereas 2 of them predict a negative label, the ensemble will predict positive (since 8 > 2).

There many ways to "aggregate" votes, with a majority vote being a common and simple approach. Alternative approaches include averaging (for regression problems) or more complex statistical methods like [Bayesian optimality](https://machinelearningmastery.com/bayes-optimal-classifier). Some of these options are implemented as part of the scikit [classifier parameters](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

The benefit of the random forest (and ensembling in general) is that individual errors are averaged out. Each single decision tree might do poorly on some examples, but across a large collection of trees, the majority of trees might perform well on all examples. The downside to ensembles is cost! If we want to ensemble 100 models, we still need to make predictions with 100 models.

Other versions of ensembling combine different model families. Nothing prevents you from ensembling a decision tree with a linear model. In fact, this increased diversity may be advantageous.

The `Ensemble` class trains a collection of models of a given type, and then makes predictions using majority vote aggregation. We train a single decision tree as well as an ensemble of decision trees on the cancer dataset we acquired from Kaggle. **You do not need to use this class directly to build tree ensembles on a practical problem. The `Ensemble` class handles creating ensembles in general. There is a pre-built ensembled tree classifier available via `sklearn.ensemble.RandomForestClassifier`**

In [ ]:
class Ensemble:
  """
  Example implementation of an ensemble with majority vote aggregation.

  @models: a list of scikit-learn models.
  """

  def __init__(self, models):
    self._models = models
    self._size = len(models)

  def fit(self, X_train, y_train):
    for i in range(self._size):
      self._models[i].fit(X_train, y_train)

  def predict(self, X_test):
    preds = []
    for i in range(self._size):
      pred_i = self._models[i].predict(X_test)
      preds.append(pred_i)
    preds = np.vstack(preds)
    agg = []
    for j in range(preds.shape[1]):
      agg.append(np.bincount(preds[:, j]).argmax())
    agg = np.array(agg)
    return agg

In [ ]:
# apply it to the cancer dataset

model = DecisionTreeClassifier(max_leaf_nodes=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f'Single test acc: {acc}')

models = [DecisionTreeClassifier(max_leaf_nodes=20) for _ in range(30)]
ensemble = Ensemble(models)
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f'Ensemble test acc: {acc}')

Single test acc: 0.9210526315789473
Ensemble test acc: 0.9385964912280702


We see the ensemble does a bit better than a single decision tree!

# Bagging: Combining Data Subsets

Suppose we have a training dataset $D$ with $N$ entries. We then generate $M$ new datasets $D_1, \ldots D_M$, each with size $N'$. We do this by sampling entries from $D$ uniformly with replacement (so that duplicated examples are possible). We then train $M$ models on each of the $M$ new datasets. For a new example, we again make $M$ predictions using each of the $M$ models and similarly aggregate the predictions by averaging or majority voting.

This technique is quite similar to ensembling, except rather than relying on model diversity, we rely on data diversity. The hypothesis is similar, that by composing together models that have individual biases from different subsets of data, we can make better predictions in aggregate. Like ensembling, bagging is quite expensive in computation and storage, as now we must store many copies of the dataset as well as a model per copy.

Here, we show a `Bagging` class that trains models of a given type on bootstrapped samples from the training dataset.  It then makes predictions using majority vote again. Again, we train a single decision tree, and then use bagging to train and combine a collection of decision trees:

In [ ]:
import copy

class Bagging:
  """
  Example implementation of bagging with majority vote aggregation.

  @model: a scikit-learn model.
  """

  def __init__(self, model, M, N_prime):
    self._model = model
    self._models = None
    self._M = M
    self._N_prime = N_prime
    self._rs = np.random.RandomState(42)

  def bootstrap(self, X, y):
    subsets = []
    N = len(X)
    for i in range(self._M):
      indices = self._rs.choice(N, self._N_prime, replace=True)
      X_i = X[indices]
      y_i = y[indices]
      subsets.append((X_i, y_i))
    return subsets

  def fit(self, X_train, y_train):
    subsets = self.bootstrap(X_train, y_train)
    models = []
    for i in range(self._M):
      X_i, y_i = subsets[i]
      model_i = copy.deepcopy(self._model)  # Q: why copy?
      model_i.fit(X_i, y_i)
      models.append(model_i)
    self._models = models

  def predict(self, X_test):
    preds = []
    for i in range(self._M):
      pred_i = self._models[i].predict(X_test)
      preds.append(pred_i)
    preds = np.vstack(preds)
    agg = []
    for j in range(preds.shape[1]):
      agg.append(np.bincount(preds[:, j]).argmax())
    agg = np.array(agg)
    return agg

In [ ]:
# apply it to the cancer dataset

model = DecisionTreeClassifier(max_leaf_nodes=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f'Single test acc: {acc}')

model = DecisionTreeClassifier(max_leaf_nodes=20)
ensemble = Bagging(model, 30, 100)
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f'Bagging test acc: {acc}')

Single test acc: 0.9473684210526315
Bagging test acc: 0.956140350877193


# Boosting: Combining Model Error

So far, we have seen methods that use independent copies of the model. Boosting presents an alternative way to combine models. First, train your first model which correctly classifies some examples and incorrectly classifies some other examples. Then, train a second model to "correct" the examples that the first model got wrong. Then, train a third model to "correct" the second model, and so on.

The "correction" process can be realized in many different ways.
- One popular method, called AdaBoost, upweights examples that the $T-1$-th model got wrong when training the $T$-th model.
- Another popular framework, called Gradient Boosting, trains the $T$-th model to predict the residual (aka difference) between the $T-1$-th model and the true label.

In either case, the final prediction is made by summing the predictions overall $T$ models. It is possible to have boosting frameworks spit out the $T$-th model as the most powerful one, rather than summing over all $T$.

In [ ]:
# It's easier to present Gradient Boosting in the regression setting.
# Let's use a different dataset from scikit-learn. This one contains
# patient features where the label is if the patient has diabetes or not.

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

dataset2 = datasets.load_diabetes()
X2 = dataset2.data
y2 = dataset2.target

scaler = StandardScaler()
y2 = scaler.fit_transform(y2[:, np.newaxis]).ravel()

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42)


print(X2_train.shape, y2_train.shape)
print(X2_test.shape, y2_test.shape)

(353, 10) (353,)
(89, 10) (89,)


In [ ]:
import copy

class GradientBoosting:
  """
  Example implementation of gradient boosting.

  @model: a scikit-learn model.
  """
  def __init__(self, model, M):
    self._model = model
    self._models = None
    self._M = M

  def fit(self, X_train, y_train):
    r_train = np.zeros_like(y_train)  # residuals
    models = []

    for i in range(self._M):
      model_i = copy.deepcopy(self._model)

      if i == 0:
        model_i.fit(X_train, y_train)
      else:
        model_i.fit(X_train, r_train)

      models.append(model_i)

      pred_i = model_i.predict(X_train)

      if i == 0:
        r_train = y_train - pred_i
      else:
        r_train = r_train - pred_i

    self._models = models

  def predict(self, X_test):
    preds = np.zeros(len(X_test))
    for i in range(self._M):
      pred_i = self._models[i].predict(X_test)
      preds += pred_i
    return preds

In [ ]:
# apply it to the diabetes dataset

model = DecisionTreeRegressor(max_leaf_nodes=20)
model.fit(X2_train, y2_train)
y2_pred = model.predict(X2_train)
mse = np.mean((y2_pred - y2_train)**2)
print(f'Single train mse: {mse}')
y2_pred = model.predict(X2_test)
mse = np.mean((y2_pred - y2_test)**2)
print(f'Single test mse: {mse}')

model = DecisionTreeRegressor(max_leaf_nodes=20)
ensemble = GradientBoosting(model, 10)
ensemble.fit(X2_train, y2_train)
y2_pred = ensemble.predict(X2_train)
mse = np.mean((y2_pred - y2_train)**2)
print(f'Boosting train mse: {mse}')
y2_pred = ensemble.predict(X2_test)
mse = np.mean((y2_pred - y2_test)**2)
print(f'Boosting test mse: {mse}')

Single train mse: 0.3417244879354889
Single test mse: 0.5824366346672816
Boosting train mse: 0.015136124358794354
Boosting test mse: 0.9836974828675755


As you notice above, gradient boosting could be over-susceptible to overfitting as it is a very expressive model. Boosting tends to work much better on larger datasets.

Gradient boosting can also be used in classification although computing residuals is slightly more technical. See this [link](https://towardsdatascience.com/gradient-boosting-classification-explained-through-python-60cc980eeb3d) for more details. Rather than implement it, we can use the existing library in scikit-learn.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Back to breast cancer dataset

model = DecisionTreeClassifier(max_leaf_nodes=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f'Single test acc: {acc}')

ensemble = GradientBoostingClassifier(n_estimators=30, max_leaf_nodes=20)
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)
acc = np.mean(y_pred == y_test)
print(f'Boosting test acc: {acc}')

Single test acc: 0.9473684210526315
Boosting test acc: 0.956140350877193


As a last note, there is a very popular library for gradient boosting decision trees called `xgboost`. You may find this library helpful in your own experiments. XGBoost is regarded as a default tool for producing SOTA results on many small to medium ML classification tasks. The library is well optimized and produces quality models consistently. As datasets grow, deep learning techniques become more relevant for achieving top performance.

In [ ]:
# xgboost requires its own data structures
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test)

param = {
    'n_estimators': 30,
    'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'binary:logistic',  # error evaluation for multiclass training
}

num_round = 20  # the number of training iterations
bst = xgb.train(param, d_train, num_round)
y_pred = bst.predict(d_test)
y_pred = np.round(y_pred).astype(int)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [06:08:20] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators", "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
acc = np.mean(y_pred == y_test)
print(f'xgboost test acc: {acc}')

xgboost test acc: 0.956140350877193
